In [1]:
import pygame
import numpy
import threading
import pyaudio
import scipy
import scipy.fftpack
import scipy.io.wavfile
import sys
import wave
from IPython.display import clear_output

pygame 2.5.2 (SDL 2.28.3, Python 3.10.14)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
import sounddevice as sd
sd.query_devices()

In [3]:
SAMPLE_RATE = 44100  # try 5000 for HD data, 48000 for realtime
DEVICE = 1
W = 500
fftsize = 512
currentCol = 0
scooter = []
overlap = 5  # 1 for raw, realtime - 8 or 16 for high-definition

In [4]:
log = {}
times = 10000

In [8]:
def update_matrix(pcm):
    global data, times, log
    logs = True
    ffty_source = scipy.fftpack.fft(pcm)  # convert WAV to FFT
    
    ffty_500 = abs(ffty_source[0:int(len(ffty_source)/2)])/500  # FFT is mirror-imaged
    log[f'500_{times}'] = ffty_500
    
    log_ffty=(scipy.fftpack.fft(ffty_500))*30-50 # if you want uniform data
    log[f'30_50_{times}'] = ffty_500
    # print("MIN:t%stMAX:t%s" % (min(ffty), max(ffty)))
    for i in range(len(ffty_500)):
        if ffty_500[i] < 0:
            ffty_500[i] = 0
        if ffty_500[i] > 255:
            ffty_500[i] = 255
    scooter.append(ffty_500)
    if len(scooter) < 6:
        return
    scooter.pop(0)
    ffty_500 = (scooter[0]+scooter[1]*2+scooter[2]*3+scooter[3]*2+scooter[4])/9
    data = numpy.roll(data, -1, 0)
    data[-1] = ffty_500[::-1]
    times += 1
    if logs:
        if times > 90_000:
            logs=False
            print('ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD')

In [14]:
log["500_10000"]

array([135.71      , 129.98525933, 113.73440039,  89.57732436,
        61.42808453,  34.10283193,  14.93741196,  16.82497071,
        23.34476999,  23.76367219,  18.85116859,  12.23589675,
         9.92546486,  12.58227975,  14.06051445,  12.41759486,
         8.70809352,   6.16760027,   7.38016406,   8.81045278,
         8.20459856,   5.95230209,   4.57475386,   6.11057322,
         7.70650489,   7.64444222,   6.08811964,   4.63644041,
         5.17365576,   6.34714111,   6.41444012,   5.19249102,
         3.8123126 ,   4.09504977,   5.21195295,   5.48276217,
         4.58392487,   3.30466041,   3.27456245,   4.22867604,
         4.60708278,   3.98910924,   2.9375365 ,   2.76848749,
         3.49374646,   3.81369128,   3.30886225,   2.50675732,
         2.63270792,   3.49006355,   3.92043858,   3.54518036,
         2.67256138,   2.2042657 ,   2.58827329,   2.94252469,
         2.79302039,   2.43050301,   2.49618253,   2.92061516,
         3.0649666 ,   2.66922349,   2.04889502,   1.98

In [11]:
log[f'500_10001']

KeyError: '500_10001'

In [ ]:
def record():
    # p = pyaudio.PyAudio()
    # inStream = p.open(format=pyaudio.paInt16,
    #                   channels=1,
    #                   rate=SAMPLE_RATE,
    #                   input_device_index=DEVICE, input=True)
    # while True:
    #     pcm = numpy.frombuffer(inStream.read(fftsize*4))
    #     #print('PCM FIRST LEN',len(pcm))
    #     update_matrix(pcm)
    p = pyaudio.PyAudio()
    inStream = p.open(format=pyaudio.paInt16, channels=1, rate=SAMPLE_RATE,
                      input_device_index=DEVICE, input=True)
    linear = [0]*fftsize
    while True:
        linear = linear[int(fftsize/overlap):]
        pcm = numpy.frombuffer(inStream.read(
            int(fftsize/overlap)), dtype=numpy.int16)
        linear = numpy.append(linear, pcm)
        update_matrix(linear)

In [ ]:
pal = [(max((x-128)*2, 0), x, min(x*2, 255)) for x in range(256)]
print( max(pal), min(pal))
data = numpy.array(numpy.zeros((W, int(fftsize/2))), dtype=int)
# data=Numeric.array(data) # for older PyGame that requires Numeric
pygame.init()  # crank up PyGame
pygame.display.set_caption("Simple Spectrograph")
screen = pygame.display.set_mode((W, fftsize/2))
world = pygame.Surface((W, fftsize/2), depth=8)  # MAIN SURFACE
world.set_palette(pal)
t_rec = threading.Thread(target=record)  # make thread for record()
t_rec.daemon = True  # daemon mode forces thread to quit with program
t_rec.start()  # launch thread
clk = pygame.time.Clock()
while 1:
    for event in pygame.event.get():  # check if we need to exit
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    pygame.surfarray.blit_array(world, data)  # place data in window
    screen.blit(world, (0, 0))
    pygame.display.flip()  # RENDER WINDOW
    clk.tick(30)  # limit to 30FPS

In [13]:

def graphFFT(pcm):
    global currentCol, data
    ffty = scipy.fftpack.fft(pcm)  # convert WAV to FFT
    ffty = abs(ffty[0:int(len(ffty)/2)])/500  # FFT is mirror-imaged
    # ffty=(scipy.log(ffty))*30-50 # if you want uniform data
    print("MIN:t%stMAX:t%s" % (min(ffty), max(ffty)))
    clear_output(wait = True)
    for i in range(len(ffty)):
        if ffty[i] < 0:
            ffty[i] = 0
        if ffty[i] > 255:
            ffty[i] = 255
    scooter.append(ffty)
    if len(scooter) < 6:
        return
    scooter.pop(0)
    ffty = (scooter[0]+scooter[1]*2+scooter[2]*3+scooter[3]*2+scooter[4])/9
    data = numpy.roll(data, -1, 0)
    data[-1] = ffty[::-1]
    currentCol += 1
    if currentCol == windowWidth:
        currentCol = 0


def record():
    p = pyaudio.PyAudio()
    inStream = p.open(format=pyaudio.paInt16, channels=1, rate=rate,
                      input_device_index=soundcard, input=True)
    linear = [0]*fftsize
    while True:
        linear = linear[int(fftsize/overlap):]
        pcm = numpy.frombuffer(inStream.read(
            int(fftsize/overlap)), dtype=numpy.int16)
        linear = numpy.append(linear, pcm)
        graphFFT(linear)


pal = [(max((x-128)*2, 0), x, min(x*2, 255)) for x in range(256)]
print( max(pal), min(pal))
data = numpy.array(numpy.zeros((windowWidth, int(fftsize/2))), dtype=int)
# data=Numeric.array(data) # for older PyGame that requires Numeric
pygame.init()  # crank up PyGame
pygame.display.set_caption("Simple Spectrograph")
screen = pygame.display.set_mode((windowWidth, fftsize/2))
world = pygame.Surface((windowWidth, fftsize/2), depth=8)  # MAIN SURFACE
world.set_palette(pal)
t_rec = threading.Thread(target=record)  # make thread for record()
t_rec.daemon = True  # daemon mode forces thread to quit with program
t_rec.start()  # launch thread
clk = pygame.time.Clock()
while 1:
    for event in pygame.event.get():  # check if we need to exit
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    pygame.surfarray.blit_array(world, data)  # place data in window
    screen.blit(world, (0, 0))
    pygame.display.flip()  # RENDER WINDOW
    clk.tick(30)  # limit to 30FPS

Exception in thread Thread-11 (record):
Traceback (most recent call last):
  File "C:\Users\chomps\.conda\envs\audios\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\chomps\.conda\envs\audios\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\chomps\AppData\Local\Temp\ipykernel_1308\3651637372.py", line 32, in record
  File "C:\Users\chomps\.conda\envs\audios\Lib\site-packages\pyaudio.py", line 608, in read
    return pa.read_stream(self._stream, num_frames, exception_on_overflow)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno -9981] Input overflowed


SystemExit: 

In [ ]:
soundcard = 1

In [ ]:
def record():
    p = pyaudio.PyAudio()
    inStream = p.open(format=pyaudio.paInt16, channels=1, rate=rate,
                      input_device_index=soundcard, input=True)
    linear = [0]*fftsize
    while True:
        linear = linear[fftsize/overlap:]
        pcm = numpy.fromstring(inStream.read(
            fftsize/overlap), dtype=numpy.int16)
        linear = numpy.append(linear, pcm)
        graphFFT(linear)

In [ ]:
record()

In [18]:
p = pyaudio.PyAudio()
inStream = p.open(format=pyaudio.paInt16, channels=1, rate=rate,
                      input_device_index=1, input=True)